## Load Data from Yfinance, NewsAPI, and Bloomberg

**Load Data (Yahoo Finance)**

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta, UTC
import time

CANONICAL_FIELDS = [
    "id",
    "report_id",
    "ticker",
    "company",
    "date",
    "source",
    "doc_type",
    "item",
    "section_type",
    "section_heading",
    "chunk_index",
    "page_start",
    "page_end",
    "text",
    "source_file",
]


def make_row(*, report_id, text, chunk_index, source, source_file, ticker=None, company=None, date=None, doc_type=None, item=None, section_type=None, section_heading=None, page_start=None, page_end=None):
    return {
        "id": f"{report_id}-{chunk_index}",
        "report_id": report_id,
        "ticker": ticker or "",
        "company": company or "",
        "date": date or "",
        "source": source,
        "doc_type": doc_type or "",
        "item": item or "",
        "section_type": section_type or "",
        "section_heading": section_heading or "",
        "chunk_index": chunk_index,
        "page_start": page_start or "",
        "page_end": page_end or "",
        "text": (text or "").strip(),
        "source_file": source_file,
    }

# Tickers to analyze
TICKERS = ["NKE", "LULU", "ATZ.CO"]
YF_MAX_ITEMS = 10  # yfinance often returns ~10
LOOKBACK_DAYS = 365


def extract_article(item: dict, ticker: str) -> dict:
    content = item.get("content", {}) or {}

    # Published time: providerPublishTime (unix) or content pubDate/displayTime
    published_dt = None
    ts = item.get("providerPublishTime")
    if ts:
        try:
            published_dt = datetime.fromtimestamp(ts, tz=UTC)
        except Exception:
            published_dt = None
    if published_dt is None:
        pub_iso = content.get("pubDate") or content.get("displayTime")
        if pub_iso:
            published_dt = pd.to_datetime(pub_iso, errors="coerce", utc=True)

    link = (
        item.get("link")
        or (item.get("canonicalUrl") or {}).get("url")
        or (item.get("clickThroughUrl") or {}).get("url")
        or (content.get("canonicalUrl") or {}).get("url")
        or (content.get("clickThroughUrl") or {}).get("url")
    )
    report_id = item.get("id") or item.get("uuid") or f"{ticker}-{int(time.time())}"
    publisher = (item.get("publisher") or (content.get("provider") or {}).get("displayName") or "").strip()
    doc_type = (item.get("type") or content.get("contentType") or "news").strip()
    heading = (content.get("title") or item.get("title") or "").strip()
    text = (content.get("summary") or content.get("description") or item.get("summary") or heading).strip()

    return make_row(
        report_id=report_id,
        text=text,
        chunk_index=0,
        source="yfinance",
        source_file=link or "",
        ticker=ticker,
        company="",
        date=str(published_dt) if published_dt is not None else "",
        doc_type=doc_type,
        section_type="news",
        section_heading=heading,
        page_start="",
        page_end="",
    )


rows = []
for tic in TICKERS:
    raw_news = (yf.Ticker(tic).news or [])[:YF_MAX_ITEMS]
    rows.extend([extract_article(item, tic) for item in raw_news])
    print(f"{tic}: fetched {len(raw_news)} raw items")
    time.sleep(1)

# Build dataframe
if rows:
    master = pd.DataFrame(rows)
    master = master.reindex(columns=CANONICAL_FIELDS)
    master["date"] = pd.to_datetime(master["date"], errors="coerce", utc=True)
    cutoff = pd.Timestamp.now(tz="UTC") - pd.Timedelta(days=LOOKBACK_DAYS)
    master = master[master["date"].isna() | (master["date"] >= cutoff)]
    master = master.drop_duplicates(subset=["section_heading", "source_file"])
    master = master.sort_values(["ticker", "date"], ascending=[True, False]).reset_index(drop=True)
else:
    master = pd.DataFrame(columns=CANONICAL_FIELDS)

print(master.head())
print()
print(master.describe(include="all"))


NKE: fetched 10 raw items
LULU: fetched 10 raw items
ATZ: fetched 0 raw items
                                       id  \
0  a9c01356-9d78-3d3f-8132-f9ac193397fb-0   
1  93b034dd-79fd-37df-9752-a3ea786023d0-0   
2  eed1f940-1dba-315b-8ed9-63073c56725b-0   
3  27d1d31f-05f4-3fe3-b142-8cdcce89dd8e-0   
4  12cc9c8f-c138-3d15-8795-91991bd0cbd1-0   

                              report_id ticker company  \
0  a9c01356-9d78-3d3f-8132-f9ac193397fb   LULU           
1  93b034dd-79fd-37df-9752-a3ea786023d0   LULU           
2  eed1f940-1dba-315b-8ed9-63073c56725b   LULU           
3  27d1d31f-05f4-3fe3-b142-8cdcce89dd8e   LULU           
4  12cc9c8f-c138-3d15-8795-91991bd0cbd1   LULU           

                       date    source doc_type item section_type  \
0 2025-12-30 00:39:00+00:00  yfinance    STORY              news   
1 2025-12-29 21:46:44+00:00  yfinance    STORY              news   
2 2025-12-29 21:44:16+00:00  yfinance    VIDEO              news   
3 2025-12-29 20:56:15+00:00  y

In [ ]:
OUTPUT_FILE = "./processed_data/final/financial_news_dataset.csv"  # optional export
master.to_csv(OUTPUT_FILE, index=False)

print(f"Data exported to {OUTPUT_FILE}")

Data exported to ./processed_data/financial_news_dataset.csv


**Import Data From News API**

In [39]:
from newsapi import NewsApiClient
from dotenv import load_dotenv
import os
from pathlib import Path

load_dotenv()
TICKERS = ["NKE", "LULU", "ATZ.TO"]
keywords = {
    #"nike": ["nke","NKE"],
    "atz": ["ATZ", "atz", "atz.co","ATZ.CO, ATZ.TO", "atz.to", "Aritzia"]
    #"lulu": ["lulu", "LULU"]
}

DATE = '2025-12-01'

api_key= os.getenv('NEWS_API_KEY')

# Init
newsapi = NewsApiClient(api_key=api_key)

**Export NewsAPI articles to CSV**


In [40]:
import pandas as pd
from pathlib import Path
import hashlib

CANONICAL_FIELDS = [
    "id",
    "report_id",
    "ticker",
    "company",
    "date",
    "source",
    "doc_type",
    "item",
    "section_type",
    "section_heading",
    "chunk_index",
    "page_start",
    "page_end",
    "text",
    "source_file",
]


def make_row(*, report_id, text, chunk_index, source, source_file, ticker=None, company=None, date=None, doc_type=None, item=None, section_type=None, section_heading=None, page_start=None, page_end=None):
    return {
        "id": f"{report_id}-{chunk_index}",
        "report_id": report_id,
        "ticker": ticker or "",
        "company": company or "",
        "date": date or "",
        "source": source,
        "doc_type": doc_type or "",
        "item": item or "",
        "section_type": section_type or "",
        "section_heading": section_heading or "",
        "chunk_index": chunk_index,
        "page_start": page_start or "",
        "page_end": page_end or "",
        "text": (text or "").strip(),
        "source_file": source_file,
    }


def stable_id(text: str, url: str) -> str:
    basis = (url or text or "").encode("utf-8")
    return hashlib.md5(basis, usedforsecurity=False).hexdigest()


def fetch_news_articles(keywords, start_date):
    rows = []
    for kw, words in keywords.items():
        query = " OR ".join(words)
        resp = newsapi.get_everything(
            q=query,
            from_param=start_date,
            language="en",
            sort_by="relevancy",
            page=1,
            page_size=100,
        )
        ticker = ""
        if kw == "nike":
            ticker = "NKE"
        if kw == "atz":
            ticker = "ATZ"
        if kw == "lulu":
            ticker = "LULU"

        for art in resp.get("articles", []):
            title = art.get("title") or ""
            text = (art.get("content") or art.get("description") or title).strip()
            if not text:
                continue
            url = art.get("url") or ""
            rid = stable_id(title, url)
            rows.append(
                make_row(
                    report_id=rid,
                    text=text,
                    chunk_index=0,
                    source="newsapi",
                    source_file=url,
                    ticker=ticker,
                    company=kw,
                    date=art.get("publishedAt") or "",
                    doc_type="news",
                    section_type="news",
                    section_heading=title,
                )
            )
    return pd.DataFrame(rows, columns=CANONICAL_FIELDS)


df_news = fetch_news_articles(keywords, DATE)

output_path = "processed_data/newsapi_articles.csv"
df_news.to_csv(output_path, index=False)

print(f"Saved {len(df_news)} rows to {output_path}")
print(df_news.head())


Saved 32 rows to processed_data/newsapi_articles.csv
                                   id                         report_id  \
0  5f922d2adea94483311ca54f04ba73b9-0  5f922d2adea94483311ca54f04ba73b9   
1  85d06731fb1c9c177393344a45b7e707-0  85d06731fb1c9c177393344a45b7e707   
2  ee8da6e3d769ca0df97ba314940fb9a5-0  ee8da6e3d769ca0df97ba314940fb9a5   
3  7ec1e9450737c4ad4481ec624aee54b0-0  7ec1e9450737c4ad4481ec624aee54b0   
4  0fed61e7630571098a8d4ec50d6cc30d-0  0fed61e7630571098a8d4ec50d6cc30d   

  ticker company                  date   source doc_type item section_type  \
0    ATZ     atz  2025-12-20T10:55:01Z  newsapi     news              news   
1    ATZ     atz  2025-12-05T17:49:06Z  newsapi     news              news   
2    ATZ     atz  2025-12-10T13:00:45Z  newsapi     news              news   
3    ATZ     atz  2025-12-12T13:38:01Z  newsapi     news              news   
4    ATZ     atz  2025-12-23T18:30:00Z  newsapi     news              news   

                           

**Analyzing Equity Research Reports from Bloomberg**

In [ ]:
"""
Done in pdf_section_extractor.py

Pulls thesis/growth/risk/valuation/earnings blocks using simple
heading heuristics, then optionally chunks text for BERT-friendly input.
"""


'\nDone in pdf_section_extractor.py\n\nPulls thesis/growth/risk/valuation/earnings blocks using simple\nheading heuristics, then optionally chunks text for BERT-friendly input.\n'

**SEC + TSEC Filings**

In [16]:
from secedgar import filings, FilingType

"""
# 8K filings for Nike and Lululemon (tickers "nke" and "lulu")
my_filings_8k = filings(cik_lookup=["nke","lulu"],
                     filing_type=FilingType.FILING_8K,
                     user_agent="Simon Kurono (simonkurono@gmail.com)")

my_filings_8k.save('./nlp/raw_data/sec_filings_8k')

# 10Q filings for Nike and Lululemon (tickers "nke" and "lulu")
my_filings_8k = filings(cik_lookup=["nke","lulu"],
                     filing_type=FilingType.FILING_10Q,
                     user_agent="Simon Kurono (simonkurono@gmail.com)")

my_filings_8k.save('./nlp/raw_data/sec_filings_10q')
"""

#Standardize tickers in SEC files

df_8k = pd.read_csv('./processed_data/final/sec_filings_extracted_8k.csv')
df_10q = pd.read_csv('./processed_data/final/sec_filings_extracted_10q.csv')
#print(df_8k.head())

"""
print(df_8k["ticker"])
print(df_8k["source_file"])
print(df_10k["ticker"])
print(df_10k["source_file"])
"""

df_8k.loc[df_8k["source_file"].str.contains("lulu", case=False, na=False), "ticker"] = "LULU"
df_8k.loc[df_8k["source_file"].str.contains("nke",  case=False, na=False), "ticker"] = "NKE"


print(df_8k["ticker"])

df_10q.loc[df_10q["source_file"].str.contains("lulu", case=False, na=False), "ticker"] = "LULU"
df_10q.loc[df_10q["source_file"].str.contains("nke", case=False, na=False), "ticker"] = "NKE"

print(df_10q["ticker"])

#save
df_8k.to_csv('./processed_data/final/sec_filings_extracted_8k.csv')
df_10q.to_csv('./processed_data/final/sec_filings_extracted_10q.csv')


0      LULU
1      LULU
2      LULU
3      LULU
4      LULU
       ... 
141     NKE
142     NKE
143     NKE
144     NKE
145     NKE
Name: ticker, Length: 146, dtype: object
0       LULU
1       LULU
2       LULU
3       LULU
4       LULU
        ... 
1974     NKE
1975     NKE
1976     NKE
1977     NKE
1978     NKE
Name: ticker, Length: 1979, dtype: object


**Compile All Data**

In [ ]:
#Convert all csvs to dataframes
import os
from pathlib import Path

print(os.getcwd())
atz_mda_df = pd.read_csv("processed_data/final/atz_mda_chunks.csv")
print(atz_mda_df.head())
print(atz_mda_df.describe())

yfinance_news_df = pd.read_csv("processed_data/final/financial_news_dataset.csv")
print(yfinance_news_df.head())
print(yfinance_news_df.describe())

newsapi_df = pd.read_csv("processed_data/final/news_data_final.csv")
print(newsapi_df.head())
print(newsapi_df.describe())

equity_report_df = pd.read_csv("processed_data/final/pdf_sections_chunks.csv")
print(equity_report_df.head())
print(equity_report_df.describe())

atz_press_release_df = pd.read_csv("processed_data/final/press_releases_chunks.csv")
print(atz_press_release_df.head())
print(atz_press_release_df.describe())

sec_8k_df = pd.read_csv("processed_data/final/sec_filings_extracted_8k.csv")
print(sec_8k_df.head())
print(sec_8k_df.describe())

sec_10q = pd.read_csv("processed_data/final/sec_filings_extracted_10q.csv")
print(sec_10q.head())
print(sec_10q.describe())



c:\Users\skurono\Desktop\SCMC\scmc-2025\nlp
                          id                report_id ticker  company  date  \
0  Aritzia-Inc-Q1-2025-MDA-0  Aritzia-Inc-Q1-2025-MDA    ATZ      NaN   NaN   
1  Aritzia-Inc-Q1-2025-MDA-1  Aritzia-Inc-Q1-2025-MDA    ATZ      NaN   NaN   
2  Aritzia-Inc-Q1-2025-MDA-2  Aritzia-Inc-Q1-2025-MDA    ATZ      NaN   NaN   
3  Aritzia-Inc-Q1-2025-MDA-3  Aritzia-Inc-Q1-2025-MDA    ATZ      NaN   NaN   
4  Aritzia-Inc-Q1-2025-MDA-4  Aritzia-Inc-Q1-2025-MDA    ATZ      NaN   NaN   

       source doc_type  item section_type  section_heading  chunk_index  \
0  mda_canada     md&a   NaN         md&a              NaN            0   
1  mda_canada     md&a   NaN         md&a              NaN            1   
2  mda_canada     md&a   NaN         md&a              NaN            2   
3  mda_canada     md&a   NaN         md&a              NaN            3   
4  mda_canada     md&a   NaN         md&a              NaN            4   

   page_start  page_end       

**Combine datasets**

In [ ]:
import pandas as pd
from pathlib import Path

CANONICAL_FIELDS = [
    'id', 'report_id', 'ticker', 'company', 'date', 'source', 'doc_type', 'item', 'section_type', 'section_heading', 'chunk_index', 'page_start', 'page_end', 'text', 'source_file'
]

base = Path('processed_data/final')
data_paths = [
    base / 'pdf_sections_chunks.csv',
    base / 'atz_mda_chunks.csv',
    base / 'sec_filings_extracted_10q.csv',
    base / 'sec_filings_extracted_8k.csv',
    base / 'press_releases_chunks.csv',
    base / 'financial_news_dataset.csv',
    base / 'news_data_final.csv',
]

frames = []
for p in data_paths:
    if not p.exists():
        print(f'[warn] missing {p}')
        continue
    master = pd.read_csv(p)
    for col in CANONICAL_FIELDS:
        if col not in master.columns:
            master[col] = ''
    master = master.reindex(columns=CANONICAL_FIELDS)
    frames.append(master)
    print(f'[info] loaded {p.name}: {len(master)} rows')

if frames:
    master = pd.concat(frames, ignore_index=True)
    master = master[master['text'].astype(str).str.len() > 0]
    master = master.drop_duplicates(subset=['text', 'source_file', 'section_heading'])
else:
    master = pd.DataFrame(columns=CANONICAL_FIELDS)

out_path = base / 'master_corpus.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)
master.to_csv(out_path, index=False)
print(f'[done] master rows={len(master)} -> {out_path}')
print(master.head())


[info] loaded pdf_sections_chunks.csv: 3178 rows
[info] loaded atz_mda_chunks.csv: 279 rows
[info] loaded sec_filings_extracted_10q.csv: 1979 rows
[info] loaded sec_filings_extracted_8k.csv: 146 rows
[info] loaded press_releases_chunks.csv: 189 rows
[info] loaded financial_news_dataset.csv: 30 rows
[info] loaded news_data_final.csv: 156 rows
[done] master rows=5955 -> processed_data/final/master_corpus.csv
                                                  id  \
0  20231103_Digital_Sales_Poised_to_Double_by_202...   
1  20231103_Digital_Sales_Poised_to_Double_by_202...   
2  20231103_Digital_Sales_Poised_to_Double_by_202...   
3  20231103_Digital_Sales_Poised_to_Double_by_202...   
4  20231103_Digital_Sales_Poised_to_Double_by_202...   

                                           report_id ticker company date  \
0  20231103_Digital_Sales_Poised_to_Double_by_202...    ATZ     NaN  NaN   
1  20231103_Digital_Sales_Poised_to_Double_by_202...    ATZ     NaN  NaN   
2  20231103_Digital_Sales

**Clean Master and ensure no mistakes**

In [ ]:
from collections import Counter
import re
import pandas as pd

master = pd.read_csv('./processed_data/backups/master_corpus.csv')
print(master["ticker"].unique())
print(master["doc_type"].unique())


print(master.duplicated("text"))
print(master.describe())
#master.to_csv('./processed_data/backups/master_corpus_no_dupes.csv')

#fix ticker


['ATZ' 'LULU' 'NKE' nan 'nike' 'lulu']
['research_note' 'md&a' '10-Q' '8-K' 'ncib_announcement'
 'earnings_release' 'agm_results' 'STORY' nan]
0       False
1       False
2       False
3       False
4       False
        ...  
5950    False
5951    False
5952    False
5953    False
5954    False
Length: 5955, dtype: bool
        Unnamed: 0  chunk_index   page_start
count  5955.000000  5799.000000  3644.000000
mean   2978.404702    15.083635     9.451976
std    1719.862629    19.333507     9.340196
min       0.000000     0.000000     1.000000
25%    1489.500000     2.000000     3.000000
50%    2979.000000     6.000000     7.000000
75%    4467.500000    24.000000    13.000000
max    5956.000000   113.000000    63.000000


In [41]:
# Sanity check
TEXT_COL = "text"   # change if needed

assert TEXT_COL in master.columns, f"Missing column: {TEXT_COL}"


# --- basic integrity ---
print("Rows:", len(master))
print("Columns:", list(master.columns))

# enforce string + fill NaNs
master[TEXT_COL] = master[TEXT_COL].astype(str).fillna("")

# --- empty / whitespace detection ---
master["is_empty"] = master[TEXT_COL].str.strip().eq("")
print("\nEmpty rows:", master["is_empty"].sum())

# --- extremely short fragments ---
master["char_len"] = master[TEXT_COL].str.len()
print("Rows < 20 chars:", (master["char_len"] < 20).sum())

# --- extremely long rows (will truncate at 512 tokens) ---
print("Rows > 2000 chars:", (master["char_len"] > 2000).sum())
print("Max length:", master["char_len"].max())

# peek at extremes
display(master.sort_values("char_len").head(5)[[TEXT_COL,"char_len"]])
display(master.sort_values("char_len").tail(5)[[TEXT_COL,"char_len"]])

# --- duplicate detection ---
dup_rows = master.duplicated(subset=[TEXT_COL]).sum()
print("\nDuplicate text rows:", dup_rows)

# --- repeated boilerplate detector (top 10 repeated snippets) ---
most_common = Counter(master[TEXT_COL]).most_common(10)
print("\nMost repeated texts:")
for t, c in most_common:
    if c > 5:
        print(f"  {c}x ->", t[:120].replace("\n"," ") + "...")

# --- garbage / artifact heuristic flags ---
def looks_like_html(s):
    return bool(re.search(r"<[^>]+>", s))

def looks_like_json(s):
    return s.strip().startswith("{") and s.strip().endswith("}")

master["html_like"] = master[TEXT_COL].apply(looks_like_html)
master["json_like"] = master[TEXT_COL].apply(looks_like_json)

print("\nPossible HTML rows:", master["html_like"].sum())
print("Possible JSON blobs:", master["json_like"].sum())

# --- language / encoding artifacts (optional heuristic) ---
master["high_symbol_ratio"] = (master[TEXT_COL].str.count(r"[^A-Za-z0-9\s.,;:$%-]") / master["char_len"].clip(lower=1)) > 0.25
print("Rows with many strange symbols:", master["high_symbol_ratio"].sum())


Rows: 5878
Columns: ['Unnamed: 0', 'id', 'report_id', 'ticker', 'company', 'date', 'source', 'doc_type', 'item', 'section_type', 'section_heading', 'chunk_index', 'page_start', 'page_end', 'text', 'source_file', 'is_empty', 'char_len', 'html_like', 'json_like', 'high_symbol_ratio']

Empty rows: 0
Rows < 20 chars: 100
Rows > 2000 chars: 656
Max length: 3094


,text,char_len
1186,% ),3
2525,1q26,4
1189,mix ),5
2110,peers,5
2046,peers,5


,text,char_len
3185,Our expansive and diverse range of fashion app...,2617
3351,"As at August 31, 2025, the fair value of the e...",2627
3398,We manage liquidity risk by continuously monit...,2674
5202,"Three Months Ended \n August 31, \n __________...",3063
5169,"Three Months Ended \n August 31, \n __________...",3094



Duplicate text rows: 1240

Most repeated texts:
  14x -> events ), with 4q net revenue ( ex - 53rd week ) expected to increase 28 - 31 %, according to guidance gross margin expa...
  13x -> expected to gain 20 - 23. 5 %, based on guidance gross margin expanded 359 bps to 43. 8 %, 250 bps ahead of consensus, d...
  13x -> post - 1q earnings outlook : aritzia could surpass 2q guidance for 19 - 22 % sales growth ( vs. analysts'20. 8 % ) and i...
  13x -> expected to increase 19 - 22 %, according to guidance gross margin expanded 320 bps to 47. 2 %, 20 bps ahead of consensu...
  13x -> to increase 3 - 7 %, based on guidance gross margin expanded 522 bps to 40. 2 %, 71 bps ahead of consensus'39. 5 %, on l...
  13x -> rise 7 - 10 %, based on guidance gross margin climbed 510 bps to 44 %, 64 bps ahead of consensus'43. 4 %, on lower markd...
  12x -> post - 3q earnings outlook : aritzia could meet the high end of 4q sales guidance of 31 % growth ( adjusting for the ext...
  12x -> rise 3 - 7 

In [20]:
#drop header/footer rows

FOOTER_SNIPPET = "additional resources : analyzer | bi » earnings calendar | evts »"

norm_text = (
    master["text"]
    .fillna("")
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
    .str.lower()
)



footer_mask = norm_text.str.contains(FOOTER_SNIPPET.lower(), regex=False)

print("Footer rows to drop:", footer_mask.sum())

master = master[~footer_mask].copy()

dup_rows = master.duplicated(subset=[TEXT_COL]).sum()
print("\nDuplicate text rows:", dup_rows)

print(master.describe())

#save master
master.to_csv('./processed_data/final/master_corpus.csv')

Footer rows to drop: 0

Duplicate text rows: 1240
        Unnamed: 0  chunk_index   page_start     char_len
count  5878.000000  5722.000000  3567.000000  5878.000000
mean   3011.858625    14.957882     9.548080  1182.447941
std    1705.664901    19.387793     9.412726   676.646121
min       0.000000     0.000000     1.000000     3.000000
25%    1547.250000     2.000000     3.000000   571.000000
50%    3017.500000     5.000000     7.000000  1399.000000
75%    4486.750000    23.000000    13.000000  1624.000000
max    5956.000000   113.000000    63.000000  3094.000000
